In [ ]:
# | default_exp application

In [ ]:
# | export

from typing import *
from typing import get_type_hints

from enum import Enum
from pathlib import Path
import json
import yaml
from copy import deepcopy
from os import environ
from datetime import datetime, timedelta
import tempfile
from fastcore.foundation import patch
from contextlib import contextmanager, asynccontextmanager
import time

import anyio
import asyncio
from asyncio import iscoroutinefunction  # do not use the version from inspect
import httpx
from fastapi import FastAPI
from fastapi import status, Depends, HTTPException, Request, Response
from fastapi.openapi.docs import get_swagger_ui_html, get_redoc_html
from fastapi.openapi.utils import get_openapi
from fastapi.responses import FileResponse, RedirectResponse
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from pydantic import Field, HttpUrl, EmailStr, PositiveInt
from pydantic.schema import schema
from pydantic.json import timedelta_isoformat

import confluent_kafka
from confluent_kafka import Producer, Consumer
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Message, KafkaError
import asyncer

import fast_kafka_api.logger

fast_kafka_api.logger.should_supress_timestamps = True

import fast_kafka_api
from fast_kafka_api.confluent_kafka import AIOProducer
from fast_kafka_api.confluent_kafka import create_missing_topics
from fast_kafka_api.asyncapi import (
    KafkaMessage,
    export_async_spec,
    _get_msg_cls_for_method,
)
from fast_kafka_api.asyncapi import (
    KafkaBroker,
    ContactInfo,
    KafkaServiceInfo,
    KafkaBrokers,
)
from fast_kafka_api.logger import get_logger
from fast_kafka_api.testing import true_after

[INFO] fast_kafka_api.asyncapi: ok


In [ ]:
# | export
logger = get_logger(__name__)

In [ ]:
# | export
logger = get_logger(__name__, level=0)

In [ ]:
logger.debug("ok")

[DEBUG] __main__: ok


In [ ]:
import pytest
import yaml
import unittest.mock
from dataclasses import dataclass

import nest_asyncio

import uvicorn
from fastapi.testclient import TestClient
from starlette.datastructures import Headers

from rich.pretty import pprint

from fast_kafka_api.confluent_kafka import create_testing_topic

In [ ]:
# | eval: false
# allows async calls in notebooks

nest_asyncio.apply()

In [ ]:
# | export
class KafkaErrorMsg(KafkaMessage):
    topic: str = Field(..., description="topic where exception occurred")
    raw_msg: Optional[bytes] = Field(None, description="raw message string")
    error: str = Field(..., description="exception triggered by the message")

In [ ]:
# | export


def _get_topic_name(
    f_or_topic: Union[Callable[[KafkaMessage], None], str], prefix: str = "on_"
):
    if isinstance(f_or_topic, str):
        topic: str = f_or_topic
    elif callable(f_or_topic):
        topic = f_or_topic.__name__
        if not topic.startswith(prefix) or len(topic) <= len(prefix):
            raise ValueError(f"Function name '{topic}' must start with {prefix}")
        topic = topic[len(prefix) :]

    return topic

In [ ]:
def on_topic_name_1():
    pass


assert _get_topic_name(on_topic_name_1) == "topic_name_1"

assert _get_topic_name("topic_name_2") == "topic_name_2"

In [ ]:
# | export


class FastKafkaAPI(FastAPI):
    def __init__(
        self,
        *,
        title: str = "FastKafkaAPI",
        contact: Optional[Dict[str, Union[str, Any]]] = None,
        kafka_brokers: Optional[Dict[str, Any]] = None,
        kafka_config: Dict[str, Any],
        root_path: Optional[Union[Path, str]] = None,
        num_partitions: Optional[int] = None,
        replication_factor: Optional[int] = None,
        **kwargs,
    ):
        self._kafka_config = kafka_config
        self.num_partitions = num_partitions
        self.replication_factor = replication_factor

        if root_path is None:
            root_path = Path(".")
        self._root_path = Path(root_path)

        if kafka_brokers is None:
            kafka_brokers = {
                "localhost": KafkaBroker(
                    url="https://localhost",
                    description="Local (dev) Kafka broker",
                    port="9092",
                )
            }
        if contact is None:
            contact = dict(
                name="author", url="https://www.google.com", email="noreply@gmail.com"
            )

        super().__init__(title=title, contact=contact, **kwargs)

        self._store: Dict[
            str,
            Dict[
                str,
                Union[
                    Callable[[KafkaMessage], Any],
                    Callable[[KafkaMessage, Message], Any],
                ],
            ],
        ] = {
            "consumers": {},
            "producers": {},
        }
        self._on_error_topic: Optional[str] = None

        contact_info = ContactInfo(**contact)  # type: ignore
        self._kafka_service_info = KafkaServiceInfo(
            title=self.title,
            version=self.version,
            description=self.description,
            contact=contact_info,
        )
        self._kafka_brokers = KafkaBrokers(brokers=kafka_brokers)

        self._confluent_producer: Optional[AIOProducer] = None

        self._asyncapi_path = self._root_path / "asyncapi"
        (self._asyncapi_path / "docs").mkdir(exist_ok=True, parents=True)
        (self._asyncapi_path / "spec").mkdir(exist_ok=True, parents=True)
        self.mount(
            "/asyncapi",
            StaticFiles(directory=self._asyncapi_path / "docs"),
            name="asyncapi",
        )

        self._is_shutting_down: bool = False
        self._kafka_consumer_tasks: List[asyncio.Task[Any]] = []
        self._kafka_producer_tasks: List[asyncio.Task[Any]] = []

        @self.get("/", include_in_schema=False)
        def redirect_root_to_asyncapi():
            return RedirectResponse("/asyncapi")

        @self.get("/asyncapi", include_in_schema=False)
        async def redirect_asyncapi_docs():
            return RedirectResponse("/asyncapi/index.html")

        @self.get("/asyncapi.yml", include_in_schema=False)
        async def download_asyncapi_yml():
            return FileResponse(self._asyncapi_path / "spec" / "asyncapi.yml")

        @self.on_event("startup")
        async def __on_startup(app=self):
            app._on_startup()

        @self.on_event("shutdown")
        async def __on_shutdown(app=self):
            await app._on_shutdown()

    async def _on_startup(self) -> None:
        raise NotImplemented

    async def _on_shutdown(self) -> None:
        raise NotImplemented

    def _add_topic(
        self,
        *,
        store_key: str,
        topic: str,
        f: Callable[[KafkaMessage], Any],
        on_error: bool,
    ):
        """Stores function `f` under key `topic` in `store`

        Params:
            store_key: either `consumers` or `producers`
            topic: the name of the topic
            f: callback function called on receiving a message for consumers or on delivery report for producers
            on_error: True for at most one producer topic used for outputting errors

        Raises:
            ValueError:
                - if store_key not one of either `consumers` or `producers`,
                - if key `topic` is already in `self._store[store_key]`, or
                - if `on_error` is already set
        """
        raise NotImplementedError

    def _register_kafka_callback(
        self,
        *,
        store_key: str,
        f_or_topic: Union[Callable[[KafkaMessage], None], str],
        prefix: str,
        on_error: bool = False,
    ) -> Callable[[KafkaMessage], None]:
        raise NotImplementedError

    def consumes(
        self, f_or_topic: Union[Callable[[KafkaMessage], None], str]
    ) -> Callable[[KafkaMessage], None]:
        """Decorator registering the callback called when a message is received in a topic.

        This function decorator is also responsible for registering topics for AsyncAPI specificiation and documentation.

        Params:
            f_or_topic: either a function or name of the topic. In the case of function, its name is
                used to infer the name of the topic ("on_topic_name" -> "topic_name")

        Returns:
            A function returning the same function

        """
        return self._register_kafka_callback(
            store_key="consumers", f_or_topic=f_or_topic, on_error=False  # type: ignore
        )

    def produces(
        self,
        f_or_topic: Union[Callable[[KafkaMessage], None], str],
        on_error: bool = False,
    ) -> Callable[[KafkaMessage], None]:
        """Decorator registering the callback called when delivery report for a produced message is received

        This function decorator is also responsible for registering topics for AsyncAPI specificiation and documentation.

        Params:
            f_or_topic: either a function or name of the topic. In the case of function, its name is
                used to infer the name of the topic ("on_topic_name" -> "topic_name")

        Returns:
            A function returning the same function

        """
        return self._register_kafka_callback(
            store_key="producers", f_or_topic=f_or_topic, on_error=on_error  # type: ignore
        )

    def produces_on_error(
        self, f_or_topic: Union[Callable[[KafkaMessage], None], str]
    ) -> Callable[[KafkaMessage], None]:
        return self._register_kafka_callback(
            store_key="producers", f_or_topic=f_or_topic, on_error=True  # type: ignore
        )

    def produce(
        self,
        topic: str,
        msg: KafkaMessage,
        on_delivery: Optional[Callable[[KafkaMessage, Message], None]] = None,
    ):
        return self.produce_raw(
            topic=topic, raw_msg=msg.json().encode("utf-8"), on_delivery=on_delivery
        )

    def produce_raw(
        self,
        topic: str,
        raw_msg: Union[str, bytes],
        on_delivery: Optional[Callable[[KafkaMessage, Message], None]] = None,
    ) -> "asyncio.Future[Any]":
        raise NotImplementedError

In [ ]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
    "bootstrap.servers": f"{kafka_server_url}:{kafka_server_port}",
    "group.id": f"{kafka_server_url}:{kafka_server_port}_group",  # ToDo: Figure out msg deletion from kafka after consuming once
    "auto.offset.reset": "earliest",
}


def create_testing_app():
    app = FastKafkaAPI(
        kafka_brokers={
            "local": {
                "url": "kafka",
                "name": "development",
                "description": "Local (dev) Kafka broker",
                "port": 9092,
            }
        },
        kafka_config=kafka_config,
        root_path="/tmp/000_FastKafkaAPI",
    )

    return app

In [ ]:
app = create_testing_app()

In [ ]:
# | export


@patch
def _add_topic(
    self: FastKafkaAPI,
    *,
    store_key: str,
    topic: str,
    f: Callable[[KafkaMessage], Any],
    on_error: bool = False,
):
    if store_key not in ["consumers", "producers"]:
        raise ValueError(
            f"store_key must be one of 'consumers', 'producers', it is '{store_key}' instead"
        )
    if on_error:
        if store_key == "consumers":
            raise ValueError(
                "`on_error` can be true only when `store_key` is `producers`"
            )
        if self._on_error_topic is not None:
            raise ValueError(
                f"`on_error` must be unique, it is already set to '{self._on_error_topic}'"
            )
        self._on_error_topic = topic

    if topic in self._store["consumers"].keys():
        raise ValueError(f"Topic '{topic}' is already in consumers.")
    if topic in self._store["producers"].keys():
        raise ValueError(f"Topic '{topic}' is already in producers.")
    self._store[store_key][topic] = f

In [ ]:
app = create_testing_app()


def f(KafkaMessage):
    pass


with pytest.raises(ValueError) as e:
    app._add_topic(store_key="random_thing", topic="topic_name_1", f=f, on_error=False)
assert (
    str(e.value)
    == "store_key must be one of 'consumers', 'producers', it is 'random_thing' instead"
), str(e.value)

with pytest.raises(ValueError) as e:
    app._add_topic(store_key="consumers", topic="topic_name_1", f=f, on_error=True)
assert (
    str(e.value) == "`on_error` can be true only when `store_key` is `producers`"
), str(e.value)

app._add_topic(store_key="producers", topic="topic_name_1", f=f, on_error=True)
assert app._store["producers"]["topic_name_1"] == f

with pytest.raises(ValueError) as e:
    app._add_topic(store_key="producers", topic="topic_name_2", f=f, on_error=True)
assert (
    str(e.value) == "`on_error` must be unique, it is already set to 'topic_name_1'"
), str(e.value)

app._add_topic(store_key="producers", topic="topic_name_2", f=f, on_error=False)
assert app._store["producers"]["topic_name_2"] == f

with pytest.raises(ValueError) as e:
    app._add_topic(store_key="producers", topic="topic_name_2", f=f, on_error=False)
assert str(e.value) == "Topic 'topic_name_2' is already in producers.", str(e.value)

with pytest.raises(ValueError) as e:
    app._add_topic(store_key="consumers", topic="topic_name_2", f=f, on_error=False)
assert str(e.value) == "Topic 'topic_name_2' is already in producers.", str(e.value)

app._add_topic(store_key="consumers", topic="topic_name_3", f=f, on_error=False)
assert app._store["consumers"]["topic_name_3"] == f

In [ ]:
# | export
def _get_first_func_arg_type(f: Callable[[Any], Any]) -> Type[Any]:
    return list(get_type_hints(f).values())[0]

In [ ]:
def some_f(x: str):
    pass


actual = _get_first_func_arg_type(some_f)
assert actual == str

In [ ]:
# | export


@patch
def _register_kafka_callback(
    self: FastKafkaAPI,
    *,
    store_key: str,
    f_or_topic: Union[Callable[[KafkaMessage], None], str],
    prefix: str = "on_",
    on_error: bool = False,
) -> Callable[[KafkaMessage], None]:

    topic = _get_topic_name(f_or_topic=f_or_topic, prefix=prefix)

    if isinstance(f_or_topic, str):

        def _decorator(
            on_topic: Callable[[KafkaMessage], Any],
            store_key: str = store_key,
            topic: str = topic,
            on_error: bool = on_error,
        ) -> Callable[[KafkaMessage], Any]:
            first_arg_type = _get_first_func_arg_type(on_topic)
            if on_error and first_arg_type != KafkaErrorMsg:
                raise ValueError(
                    f"The first argument of a decorator handling errors must be KafkaErrorMsg, it is '{first_arg_type}' instead"
                )
            self._add_topic(
                store_key=store_key, topic=topic, f=on_topic, on_error=on_error
            )
            return on_topic

        return _decorator  # type: ignore
    elif callable(f_or_topic):
        first_arg_type = _get_first_func_arg_type(f_or_topic)
        if on_error and first_arg_type != KafkaErrorMsg:
            raise ValueError(
                f"The first argument of a decorator handling errors must be KafkaErrorMsg, it is '{first_arg_type}' instead"
            )
        self._add_topic(
            store_key=store_key, topic=topic, f=f_or_topic, on_error=on_error
        )
        return f_or_topic
    else:
        raise ValueError(
            f"Called on object of type {type(f_or_topic)}, should be called on 'str' or 'callable' only."
        )

In [ ]:
# ToDo: Write tests for on_error topic with first argument as wrong arg type

In [ ]:
app = create_testing_app()


def on_topic_1(msg: KafkaMessage):
    pass


actual = app._register_kafka_callback(store_key="consumers", f_or_topic=on_topic_1)
assert actual == on_topic_1
assert app._store["consumers"]["topic_1"] == on_topic_1

decorator = app._register_kafka_callback(store_key="consumers", f_or_topic="topic_2")


@decorator
def some_callback(msg: KafkaMessage):
    pass


assert app._store["consumers"]["topic_2"] == some_callback

In [ ]:
class MyInfo(KafkaMessage):
    mobile: str = Field(..., example="+385987654321")
    name: str = Field(..., example="James Bond")


class MyMsgUrl(KafkaMessage):
    info: MyInfo = Field(..., example=dict(mobile="+385987654321", name="James Bond"))
    url: HttpUrl = Field(..., example="https://sis.gov.uk/agents/007")


class MyMsgEmail(KafkaMessage):
    msg_url: MyMsgUrl = Field(
        ...,
        example=dict(
            info=dict(mobile="+385987654321", name="James Bond"),
            url="https://sis.gov.uk/agents/007",
        ),
    )
    email: EmailStr = Field(..., example="agent-007@sis.gov.uk")


def setup_testing_app():
    app = create_testing_app()

    @app.consumes("my_topic_1")
    def on_my_topic_one(msg: MyMsgUrl):
        logger.debug(f"on_my_topic_one({msg=})")

    @app.consumes
    async def on_my_topic_2(msg: MyMsgEmail):
        logger.debug(f"on_my_topic_2({msg=})")

    with pytest.raises(ValueError) as e:

        @app.consumes
        def my_topic_3(msg: MyMsgEmail):
            raise NotImplemented

    with pytest.raises(ValueError) as e:

        @app.produces
        async def on_my_topic_1(msg: MyMsgUrl, kafka_msg: Message):
            pass

    @app.produces
    def on_my_topic_3(msg: MyMsgUrl, kafka_msg: Message):
        logger.debug(f"on_my_topic_3({msg=}, {kafka_msg=})")

    @app.produces
    def on_my_topic_4(msg: MyMsgEmail, kafka_msg: Message):
        logger.debug(f"on_my_topic_4({msg=}, {kafka_msg=})")

    @app.produces_on_error
    async def on_my_topic_error(raw_msg: KafkaErrorMsg, kafka_err: KafkaError):
        logger.warning(f"on_error({raw_msg=}, {kafka_err=})")

    return app

In [ ]:
app = setup_testing_app()

pprint(app._store)
assert set(app._store["consumers"].keys()) == set(["my_topic_1", "my_topic_2"])
assert set(app._store["producers"].keys()) == set(
    ["my_topic_3", "my_topic_4", "my_topic_error"]
)

print(f"{app._kafka_service_info=}")

print(f"{app._kafka_brokers=}")

{
│   'consumers': {
│   │   'my_topic_1': <function setup_testing_app.<locals>.on_my_topic_one at 0x7ff6b6f1f430>,
│   │   'my_topic_2': <function setup_testing_app.<locals>.on_my_topic_2 at 0x7ff6b73bd670>
│   },
│   'producers': {
│   │   'my_topic_3': <function setup_testing_app.<locals>.on_my_topic_3 at 0x7ff6b6f178b0>,
│   │   'my_topic_4': <function setup_testing_app.<locals>.on_my_topic_4 at 0x7ff6b6f17790>,
│   │   'my_topic_error': <function setup_testing_app.<locals>.on_my_topic_error at 0x7ff6b6f17940>
│   }
}

app._kafka_service_info=KafkaServiceInfo(title='FastKafkaAPI', version='0.1.0', description='', contact=ContactInfo(name='author', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com'))
app._kafka_brokers=KafkaBrokers(brokers={'local': KafkaBroker(url='kafka', description='Local (dev) Kafka broker', port='9092', protocol='kafka', security=None)})


In [ ]:
# | export


async def _consumer_pooling_step(
    *,
    async_poll_f: Callable[[float], Optional[Message]],
    timeout: float,
    topic: str,
    on_event_callback: Callable[[KafkaMessage], None],
    on_error_callback: Optional[Callable[[KafkaErrorMsg], None]] = None,
    msg_type: Type[KafkaMessage],
) -> None:
    logger.debug("_consumer_pooling_step()")
    #     print(f"{iscoroutinefunction(async_poll_f)=}")
    if not iscoroutinefunction(async_poll_f):
        raise ValueError(
            f"async_poll_f ({async_poll_f}) must be coroutine, but it isn't."
        )
    if not iscoroutinefunction(on_event_callback):
        raise ValueError(
            f"on_event_callback ({on_event_callback}) must be coroutine, but it isn't."
        )

    try:
        # we convert the blocking poll() function into asynchronous one, while executing poll() in a worker thread
        msg = await async_poll_f(timeout=timeout)  # type: ignore
        if msg is None:
            logger.debug(
                f"consumers_async_loop({topic=}): no messages for the topic {topic} due to no message available."
            )
        elif msg.error() is not None:
            logger.warning(
                f"consumers_async_loop({topic=}): no messages for the topic {topic} due to error: {msg.error()}"
            )
            if on_error_callback is not None:
                kafka_err_msg = KafkaErrorMsg(
                    topic=topic,
                    raw_msg=None,
                    error=msg.error(),
                )
                on_error_callback(kafka_err_msg)

        else:
            #             msg_type = _get_first_func_arg_type(on_event_callback)
            logger.debug(
                f"consumers_async_loop({topic=}): message received for the topic {topic}: {msg.value()}, {on_event_callback}, {msg_type=}"
            )
            msg_object = msg_type.parse_raw(msg.value().decode("utf-8"))
            logger.debug(
                f"consumers_async_loop({topic=}): calling {on_event_callback}({msg_object})"
            )
            await on_event_callback(msg_object)

    except Exception as e:
        import traceback

        logger.warning(
            f"consumers_async_loop({topic=}): Exception in inner try raised: {e}"
            + "\n"
            + traceback.format_exc()
        )

        if on_error_callback is not None:
            kafka_err_msg = KafkaErrorMsg(
                topic=topic,
                raw_msg=msg.value().decode("utf-8"),
                error=str(e),
            )
            on_error_callback(kafka_err_msg)

In [ ]:
on_event_callback = unittest.mock.AsyncMock(return_value=None)
on_error_callback = unittest.mock.Mock(return_value=None)

timeout = 0.1

async_poll_f = unittest.mock.AsyncMock(return_value=None)
asyncio.run(
    _consumer_pooling_step(
        async_poll_f=async_poll_f,
        timeout=timeout,
        topic="my_topic",
        on_event_callback=on_event_callback,
        on_error_callback=on_error_callback,
        msg_type=MyMsgUrl,
    )
)
async_poll_f.assert_called_once_with(timeout=timeout)
on_event_callback.assert_not_called()
on_error_callback.assert_not_called()

[DEBUG] __main__: _consumer_pooling_step()
[DEBUG] __main__: consumers_async_loop(topic='my_topic'): no messages for the topic my_topic due to no message available.


In [ ]:
on_event_callback = unittest.mock.AsyncMock(return_value=None)
on_error_callback = unittest.mock.Mock(return_value=None)

timeout = 0.1

m = unittest.mock.MagicMock()
m.error = unittest.mock.Mock(return_value="some error occurred")
m.value = unittest.mock.Mock(return_value=None)
async_poll_f = unittest.mock.AsyncMock(return_value=m)

asyncio.run(
    _consumer_pooling_step(
        async_poll_f=async_poll_f,
        timeout=timeout,
        topic="my_topic",
        on_event_callback=on_event_callback,
        on_error_callback=on_error_callback,
        msg_type=MyMsgUrl,
    )
)
async_poll_f.assert_called_once_with(timeout=timeout)
on_event_callback.assert_not_called()
on_error_callback.assert_called_once_with(
    KafkaErrorMsg(topic="my_topic", raw_msg=None, error="some error occurred")
)

[DEBUG] __main__: _consumer_pooling_step()
[WARNING] __main__: consumers_async_loop(topic='my_topic'): no messages for the topic my_topic due to error: some error occurred


In [ ]:
on_event_callback = unittest.mock.AsyncMock(return_value=None)
# async def on_event_callback(my_msg_url: MyMsgUrl) -> None:
#     await _on_event_callback(my_msg_url)

on_error_callback = unittest.mock.Mock(return_value=None)

timeout = 0.1

msg = MyMsgUrl(
    info=MyInfo(mobile=385999999999, name="Marko"),
    url="https://www.acme.com",
)
m = unittest.mock.MagicMock()
m.error = unittest.mock.Mock(return_value=None)
m.value = unittest.mock.Mock(return_value=msg.json().encode("utf-8"))
async_poll_f = unittest.mock.AsyncMock(return_value=m)

asyncio.run(
    _consumer_pooling_step(
        async_poll_f=async_poll_f,
        timeout=timeout,
        topic="my_topic",
        on_event_callback=on_event_callback,
        on_error_callback=on_error_callback,
        msg_type=MyMsgUrl,
    )
)
async_poll_f.assert_called_once_with(timeout=timeout)
on_event_callback.assert_called_once_with(msg)
on_error_callback.assert_not_called()

[DEBUG] __main__: _consumer_pooling_step()
[DEBUG] __main__: consumers_async_loop(topic='my_topic'): message received for the topic my_topic: b'{"info": {"mobile": "385999999999", "name": "Marko"}, "url": "https://www.acme.com"}', <AsyncMock id='140697607526576'>, msg_type=<class '__main__.MyMsgUrl'>
[DEBUG] __main__: consumers_async_loop(topic='my_topic'): calling <AsyncMock id='140697607526576'>(info=MyInfo(mobile='385999999999', name='Marko') url=HttpUrl('https://www.acme.com', ))


In [ ]:
on_event_callback = unittest.mock.AsyncMock(return_value=None)
on_error_callback = unittest.mock.Mock(return_value=None)

timeout = 0.1

msg = MyMsgUrl(
    info=MyInfo(mobile=385999999999, name="Marko"),
    url="https://www.acme.com",
)
m = unittest.mock.MagicMock()
m.error = unittest.mock.Mock(return_value=None)
m.value = unittest.mock.Mock(return_value=msg.json().encode("utf-8"))
async_poll_f = unittest.mock.AsyncMock(return_value=m)

asyncio.run(
    _consumer_pooling_step(
        async_poll_f=async_poll_f,
        timeout=timeout,
        topic="my_topic",
        on_event_callback=on_event_callback,
        on_error_callback=on_error_callback,
        msg_type=MyInfo,
    )
)
async_poll_f.assert_called_once_with(timeout=timeout)
on_event_callback.assert_not_called()
on_error_callback.assert_called_once_with(
    KafkaErrorMsg(
        topic="my_topic",
        raw_msg='{"info": {"mobile": "385999999999", "name": "Marko"}, "url": "https://www.acme.com"}',
        error="2 validation errors for MyInfo\nmobile\n  field required (type=value_error.missing)\nname\n  field required (type=value_error.missing)",
    )
)

[DEBUG] __main__: _consumer_pooling_step()
[DEBUG] __main__: consumers_async_loop(topic='my_topic'): message received for the topic my_topic: b'{"info": {"mobile": "385999999999", "name": "Marko"}, "url": "https://www.acme.com"}', <AsyncMock id='140697608312288'>, msg_type=<class '__main__.MyInfo'>
[WARNING] __main__: consumers_async_loop(topic='my_topic'): Exception in inner try raised: 2 validation errors for MyInfo
mobile
  field required (type=value_error.missing)
name
  field required (type=value_error.missing)
Traceback (most recent call last):
  File "<ipython-input-23-4781567d70ff>", line 48, in _consumer_pooling_step
    msg_object = msg_type.parse_raw(msg.value().decode("utf-8"))
  File "pydantic/main.py", line 549, in pydantic.main.BaseModel.parse_raw
  File "pydantic/main.py", line 526, in pydantic.main.BaseModel.parse_obj
  File "pydantic/main.py", line 342, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 2 validation errors for MyInfo
mobile
 

In [ ]:
# | export


async def _consumers_async_loop(
    *,
    topic: str,
    on_event_callback: Callable[[KafkaMessage], Any],
    is_shutting_down_f: Callable[[], bool],
    config: Dict[str, str],
    timeout: float = 1.0,
    app: FastKafkaAPI,
):
    logger.info(f"consumers_async_loop({topic=}, {config=}, {timeout=}) starting.")
    try:
        c: Consumer = None

        c = Consumer(config)
        logger.info(
            f"consumers_async_loop({topic=}): Kafka Consumer for topic created."
        )

        c.subscribe([topic])
        logger.info(
            f"consumers_async_loop({topic=}): Kafka Consumer subscribed to topic."
        )

        # we convert the blocking poll() function into asynchronous one (it executes poll() in a worker thread)
        async_poll_f = asyncer.asyncify(c.poll)

        # convert on_event_callback to coroutine if needed
        async_on_event_callback = on_event_callback
        if not iscoroutinefunction(async_on_event_callback):
            async_on_event_callback = asyncer.asyncify(async_on_event_callback)
        msg_type = _get_first_func_arg_type(on_event_callback)

        def on_error_callback(error_msg: KafkaErrorMsg, app=app) -> None:
            app.produce(topic=app._on_error_topic, msg=error_msg)

        while True:
            if is_shutting_down_f():
                logger.info(f"consumers_async_loop({topic=}) shutting down...")
                break

            await _consumer_pooling_step(
                async_poll_f=async_poll_f,
                timeout=timeout,
                topic=topic,
                on_event_callback=async_on_event_callback,
                on_error_callback=on_error_callback,
                msg_type=msg_type,
            )

    except Exception as e:
        logger.error(
            f"consumers_async_loop({topic=}): Exception in outer try raised: {e}"
        )

    finally:
        if c is not None:
            c.close()
            logger.info(f"consumers_async_loop({topic=}): Kafka Consumer closed.")

    logger.info(f"consumers_async_loop({topic=}) exiting.")

In [ ]:
my_info = info = MyInfo(mobile=385999999999, name="Marko")
my_url_msg = MyMsgUrl(
    info=my_info,
    url="https://www.acme.com",
)
my_email_msg = MyMsgEmail(
    msg_url=my_url_msg,
    email="marko@acme.com",
)


def _create_mock(x):
    if x is None:
        return None
    else:
        mock = unittest.mock.Mock()
        mock.error = unittest.mock.Mock(return_value=x[1])
        mock.value = unittest.mock.Mock(return_value=x[0])
        return mock


test_messages = [
    None,
    (None, "some error occured"),
    (my_url_msg.json().encode("utf-8"), None),
    (my_email_msg.json().encode("utf-8"), None),
]
test_messages = [_create_mock(x) for x in test_messages]


def get_poll_f(test_messages):
    counter = {}  # {"i": -1}

    def f(self, timeout: float, test_messages=test_messages, counter=counter):
        if self not in counter:
            counter[self] = -1
        counter[self] = counter[self] + 1
        if len(test_messages) > counter[self]:
            retval = test_messages[counter[self]]
        else:
            retval = None

        if retval is None and timeout is not None:
            time.sleep(timeout)

        return retval

    return f


poll = get_poll_f(test_messages)
for _ in range(6):
    m = poll("self", timeout=0.1)
    print(f"{m=}")
    if m is not None:
        print(f"{m.error()=}")
        print(f"{m.value()=}")

m=None
m=<Mock id='140697607648352'>
m.error()='some error occured'
m.value()=None
m=<Mock id='140697607648880'>
m.error()=None
m.value()=b'{"info": {"mobile": "385999999999", "name": "Marko"}, "url": "https://www.acme.com"}'
m=<Mock id='140697607648928'>
m.error()=None
m.value()=b'{"msg_url": {"info": {"mobile": "385999999999", "name": "Marko"}, "url": "https://www.acme.com"}, "email": "marko@acme.com"}'
m=None
m=None


In [ ]:
@contextmanager
def patch_consumer(test_messages: List[Optional[unittest.mock.Mock]] = test_messages):
    global Consumer
    org_consumer = Consumer

    class Wrapper:
        def __init__(self, *args, **kwargs):
            self._wrapped_obj = org_consumer(*args, **kwargs)

        def __getattr__(self, attr):
            if attr in self.__dict__:
                return getattr(self, attr)
            return getattr(self._wrapped_obj, attr)

    try:
        Consumer = Wrapper
        setattr(Wrapper, "poll", get_poll_f(test_messages))
        yield
    finally:
        Consumer = org_consumer


with patch_consumer():
    c1 = Consumer(kafka_config)
    c2 = Consumer(kafka_config)
    msgs = {c: [c.poll(timeout=0.1) for _ in range(6)] for c in [c1, c2]}
    print(msgs)
    assert msgs[c1] == msgs[c2]

{<__main__.patch_consumer.<locals>.Wrapper object>: [None, <Mock id='140697607648352'>, <Mock id='140697607648880'>, <Mock id='140697607648928'>, None, None], <__main__.patch_consumer.<locals>.Wrapper object>: [None, <Mock id='140697607648352'>, <Mock id='140697607648880'>, <Mock id='140697607648928'>, None, None]}


In [ ]:
# @contextmanager
# def patch_producer():
#     global Producer
#     org_producer = Producer
#     class Wrapper():
#         def __init__(self, *args, **kwargs):
#             self._wrapped_obj = org_producer(*args, **kwargs)
#         def __getattr__(self, attr):
#             if attr in self.__dict__:
#                 return getattr(self, attr)
#             return getattr(self._wrapped_obj, attr)

#     try:
#         Producer = Wrapper
#         mock = unittest.mock.Mock(return_value=None)
#         setattr(Wrapper, "produce", mock)
#         yield mock
#     finally:
#         Producer = org_producer

# with patch_producer() as produce_mock:
#     p = Producer(kafka_config)

#     p.produce("davor")

#     produce_mock.called_once_with("davor")

In [ ]:
app = setup_testing_app()

_on_event_callback = unittest.mock.AsyncMock(return_value=None)


async def on_event_callback(msg: MyMsgUrl) -> None:
    await _on_event_callback(msg)


_on_event_callback = unittest.mock.Mock(return_value=None)


def on_event_callback(msg: MyMsgUrl) -> None:
    return _on_event_callback(msg)


# def on_delivery(app, topic, msg):
#     logger.info(f"on_delivery({topic=}, {msg=})")

expected_produce_calls = [
    unittest.mock.call(
        topic="my_topic_error",
        raw_msg=KafkaErrorMsg(
            topic="my_topic",
            raw_msg=None,
            error="some error occured",
        )
        .json()
        .encode("utf-8"),
        on_delivery=None,
    ),
    unittest.mock.call(
        topic="my_topic_error",
        raw_msg=KafkaErrorMsg(
            topic="my_topic",
            raw_msg=my_email_msg.json().encode("utf-8"),
            error="2 validation errors for MyMsgUrl\ninfo\n  field required (type=value_error.missing)\nurl\n  field required (type=value_error.missing)",
        )
        .json()
        .encode("utf-8"),
        on_delivery=None,
    ),
]


async def test_me():
    with patch_consumer():
        with unittest.mock.patch.object(
            FastKafkaAPI, "produce_raw", return_value=None
        ) as produce_mock:
            await _consumers_async_loop(
                topic="my_topic",
                on_event_callback=on_event_callback,
                config=kafka_config,
                timeout=0.1,
                app=app,
                is_shutting_down_f=true_after(2),
            )
            print(produce_mock.mock_calls)
            _on_event_callback.assert_called_once_with(my_url_msg)
            produce_mock.assert_has_calls(expected_produce_calls)


asyncio.run(test_me())

[INFO] __main__: consumers_async_loop(topic='my_topic', config={'bootstrap.servers': 'davor-kafka:9092', 'group.id': 'davor-kafka:9092_group', 'auto.offset.reset': 'earliest'}, timeout=0.1) starting.
[INFO] __main__: consumers_async_loop(topic='my_topic'): Kafka Consumer for topic created.
[INFO] __main__: consumers_async_loop(topic='my_topic'): Kafka Consumer subscribed to topic.
[DEBUG] __main__: _consumer_pooling_step()
[DEBUG] __main__: consumers_async_loop(topic='my_topic'): no messages for the topic my_topic due to no message available.
[DEBUG] __main__: _consumer_pooling_step()
[WARNING] __main__: consumers_async_loop(topic='my_topic'): no messages for the topic my_topic due to error: some error occured
[DEBUG] __main__: _consumer_pooling_step()
[DEBUG] __main__: consumers_async_loop(topic='my_topic'): message received for the topic my_topic: b'{"info": {"mobile": "385999999999", "name": "Marko"}, "url": "https://www.acme.com"}', <function asyncify.<locals>.wrapper>, msg_type=<c

In [ ]:
# | export


def consumers_async_loop(
    *,
    app: FastKafkaAPI,
    timeout: float = 1.0,
    is_shutting_down_f: Callable[[], bool],
):
    config: Dict[str, str] = app._kafka_config

    # Used to create missing topics
    topics = sorted(
        set(app._store["consumers"].keys()).union(set(app._store["producers"].keys()))
    )

    kafka_admin = AdminClient(config)
    logger.info(f"consumers_async_loop(): Kafka admin created {kafka_admin}.")
    create_missing_topics(
        admin=kafka_admin,
        topic_names=topics,
        num_partitions=app.num_partitions,
        replication_factor=app.replication_factor,
    )
    logger.info(f"consumers_async_loop(): Kafka topics {topics} created if needed.")

    tx = [
        asyncio.create_task(
            _consumers_async_loop(
                app=app,
                topic=topic,
                on_event_callback=on_event_callback,  # type: ignore
                config=config,
                timeout=timeout,
                is_shutting_down_f=is_shutting_down_f,
            )
        )
        for topic, on_event_callback in app._store["consumers"].items()
    ]

    return tx

In [ ]:
app = setup_testing_app()

expected_produce_calls = [
    unittest.mock.call(
        topic="my_topic_error",
        raw_msg=KafkaErrorMsg(
            topic="my_topic_2", raw_msg=None, error="some error occured"
        )
        .json()
        .encode("utf-8"),
        on_delivery=None,
    ),
    unittest.mock.call(
        topic="my_topic_error",
        raw_msg=b'{"topic": "my_topic_1", "raw_msg": null, "error": "some error occured"}',
        on_delivery=None,
    ),
    unittest.mock.call(
        topic="my_topic_error",
        raw_msg=b'{"topic": "my_topic_2", "raw_msg": "{\\"info\\": {\\"mobile\\": \\"385999999999\\", \\"name\\": \\"Marko\\"}, \\"url\\": \\"https://www.acme.com\\"}", "error": "2 validation errors for MyMsgEmail\\nmsg_url\\n  field required (type=value_error.missing)\\nemail\\n  field required (type=value_error.missing)"}',
        on_delivery=None,
    ),
    unittest.mock.call(
        topic="my_topic_error",
        raw_msg=b'{"topic": "my_topic_1", "raw_msg": "{\\"msg_url\\": {\\"info\\": {\\"mobile\\": \\"385999999999\\", \\"name\\": \\"Marko\\"}, \\"url\\": \\"https://www.acme.com\\"}, \\"email\\": \\"marko@acme.com\\"}", "error": "2 validation errors for MyMsgUrl\\ninfo\\n  field required (type=value_error.missing)\\nurl\\n  field required (type=value_error.missing)"}',
        on_delivery=None,
    ),
]


async def test_me():
    with patch_consumer():
        with unittest.mock.patch.object(
            FastKafkaAPI, "produce_raw", return_value=None
        ) as produce_mock:
            tx = consumers_async_loop(
                app=app,
                timeout=0.1,
                is_shutting_down_f=true_after(2),
            )
            tx = [await t for t in tx]
            print(f"{tx=}")
            print(produce_mock.mock_calls)
            assert len(produce_mock.mock_calls) == 4
            for call in expected_produce_calls:
                assert call in produce_mock.mock_calls


asyncio.run(test_me())

[INFO] __main__: consumers_async_loop(): Kafka admin created <confluent_kafka.admin.AdminClient object>.
[DEBUG] fast_kafka_api.confluent_kafka: create_missing_topics(['my_topic_1', 'my_topic_2', 'my_topic_3', 'my_topic_4', 'my_topic_error']): existing_topics=['my_topic_error', 'something_random_2', 'my_topic_1', 'something_random', '__consumer_offsets', 'B', 'C', 'training_data', 'prediction_status', 'realitime_data', 'training_request', 'training_status', 'model_metrics', 'training_model_status', 'prediction', 'prediction_request', 'training_data_status', 'my_topic_4', 'my_topic_2', 'error', 'my_topic_3', 'something_random_3'], num_partitions=1, replication_factor=1
[INFO] __main__: consumers_async_loop(): Kafka topics ['my_topic_1', 'my_topic_2', 'my_topic_3', 'my_topic_4', 'my_topic_error'] created if needed.
[INFO] __main__: consumers_async_loop(topic='my_topic_1', config={'bootstrap.servers': 'davor-kafka:9092', 'group.id': 'davor-kafka:9092_group', 'auto.offset.reset': 'earliest

In [ ]:
# | export


@patch
def _on_startup(self: FastKafkaAPI) -> None:
    export_async_spec(
        consumers=self._store["consumers"],  # type: ignore
        producers=self._store["producers"],  # type: ignore
        kafka_brokers=self._kafka_brokers,
        kafka_service_info=self._kafka_service_info,
        asyncapi_path=self._asyncapi_path,
    )

    self._is_shutting_down = False

    def is_shutting_down_f(self: FastKafkaAPI = self) -> bool:
        return self._is_shutting_down

    self._kafka_consumer_tasks = consumers_async_loop(
        app=self,
        is_shutting_down_f=is_shutting_down_f,
    )

    self._confluent_producer = AIOProducer(self._kafka_config)
    logger.info("AIOProducer created.")


@patch
async def _on_shutdown(self: FastKafkaAPI) -> None:
    self._is_shutting_down = True
    await asyncio.wait(self._kafka_consumer_tasks)
    self._confluent_producer.close()  # type: ignore
    logger.info("AIOProducer closed.")

    self._is_shutting_down = False

In [ ]:
@asynccontextmanager
async def start_test_app():
    app = setup_testing_app()

    try:

        app._on_startup()

        yield app

    finally:
        await app._on_shutdown()

In [ ]:
expected = """asyncapi: 2.5.0
info:
  title: FastKafkaAPI
  version: 0.1.0
  description: ''
  contact:
    name: author
    url: https://www.google.com
    email: noreply@gmail.com
servers:
  local:
    url: kafka
    description: Local (dev) Kafka broker
    protocol: kafka
    variables:
      port:
        default: '9092'
channels:
  my_topic_1:
    subscribe:
      message:
        $ref: '#/components/messages/MyMsgUrl'
  my_topic_2:
    subscribe:
      message:
        $ref: '#/components/messages/MyMsgEmail'
  my_topic_3:
    publish:
      message:
        $ref: '#/components/messages/MyMsgUrl'
  my_topic_4:
    publish:
      message:
        $ref: '#/components/messages/MyMsgEmail'
  my_topic_error:
    publish:
      message:
        $ref: '#/components/messages/KafkaErrorMsg'
components:
  messages:
    MyMsgUrl:
      payload:
        title: MyMsgUrl
        type: object
        properties:
          info:
            title: Info
            example:
              mobile: '+385987654321'
              name: James Bond
            allOf:
            - $ref: '#/components/schemas/MyInfo'
          url:
            title: Url
            example: https://sis.gov.uk/agents/007
            minLength: 1
            maxLength: 2083
            format: uri
            type: string
        required:
        - info
        - url
        example:
          info:
            mobile: '+385987654321'
            name: James Bond
          url: https://sis.gov.uk/agents/007
    MyMsgEmail:
      payload:
        title: MyMsgEmail
        type: object
        properties:
          msg_url:
            title: Msg Url
            example:
              info:
                mobile: '+385987654321'
                name: James Bond
              url: https://sis.gov.uk/agents/007
            allOf:
            - $ref: '#/components/messages/MyMsgUrl'
          email:
            title: Email
            example: agent-007@sis.gov.uk
            type: string
            format: email
        required:
        - msg_url
        - email
        example:
          msg_url:
            info:
              mobile: '+385987654321'
              name: James Bond
            url: https://sis.gov.uk/agents/007
          email: agent-007@sis.gov.uk
    KafkaErrorMsg:
      payload:
        title: KafkaErrorMsg
        type: object
        properties:
          topic:
            title: Topic
            description: topic where exception occurred
            type: string
          raw_msg:
            title: Raw Msg
            description: raw message string
            format: binary
            type: string
          error:
            title: Error
            description: exception triggered by the message
            type: string
        required:
        - topic
        - error
  schemas:
    MyInfo:
      payload:
        title: MyInfo
        type: object
        properties:
          mobile:
            title: Mobile
            example: '+385987654321'
            type: string
          name:
            title: Name
            example: James Bond
            type: string
        required:
        - mobile
        - name
  securitySchemes: {}
"""

In [ ]:
async def test_me():
    async with start_test_app() as app:
        client = TestClient(app)
        response = client.get("/asyncapi.yml")
        assert response.status_code == 200
        assert yaml.safe_load(response.text) == yaml.safe_load(
            expected
        ), f"{yaml.safe_load(response.text)} != {yaml.safe_load(expected)}"
        await asyncio.sleep(2)


asyncio.run(test_me())

print("ok")

[INFO] fast_kafka_api.asyncapi: Async specifications generated at: '/tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api.asyncapi: Async docs generated at '/tmp/000_FastKafkaAPI/asyncapi/docs'
[INFO] fast_kafka_api.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag /tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml @asyncapi/html-template -o /tmp/000_FastKafkaAPI/asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/000_FastKafkaAPI/asyncapi/docs.


[INFO] __main__: consumers_async_loop(): Kafka admin created <confluent_kafka.admin.AdminClient object>.
[DEBUG] fast_kafka_api.confluent_kafka: create_missing_topics(['my_topic_1', 'my_topic_2', 'my_topic_3', 'my_topic_4', 'my_topic_error']): existing_topics=['my_topic_error', 'something_random_2', 'my_topic_1', 'something_random', '__consumer_offsets', 'B', 'C', 'training_data', 'prediction_status', 'realitime_data', 'training_request', 'training_status', 'model_metrics', 'training_mo

In [ ]:
# | export


@patch
def produce_raw(
    self: FastKafkaAPI,
    topic: str,
    raw_msg: Union[str, bytes],
    on_delivery: Optional[Callable[[KafkaMessage, Message], None]] = None,
) -> "asyncio.Future[Any]":

    if isinstance(raw_msg, str):
        raw_msg = raw_msg.encode("utf-8")

    if on_delivery is None:
        on_delivery = self._store["producers"][topic]  # type: ignore

    if iscoroutinefunction(on_delivery):
        raise ValueError("coroutines not supported for callbacks yet")

    p: AIOProducer = self._confluent_producer  # type: ignore

    def _delivery_report(
        kafka_err: KafkaError,
        kafka_msg: Message,
        self=self,
        topic=topic,
        raw_msg=raw_msg,
        on_delivery=on_delivery,
    ):
        msg_cls: KafkaMessage
        if kafka_err is not None:
            logger.info(f"produce_raw() {topic=} {raw_msg=} delivery error")
            if self._on_error_topic is not None:
                on_error = self._store["producers"][self._on_error_topic]
                msg_cls = _get_msg_cls_for_method(on_error)
                on_error(
                    msg_cls("Message delivery failed: {}".format(kafka_err)), kafka_err  # type: ignore
                )
        else:
            logger.info(f"produce_raw() {topic=} {raw_msg=} delivered")
            msg_cls = _get_msg_cls_for_method(on_delivery)
            on_delivery(msg_cls.parse_raw(raw_msg), kafka_msg)

    return p.produce(topic, raw_msg, on_delivery=_delivery_report)

In [ ]:
raw_msg = (
    MyMsgUrl(
        info=dict(mobile="+385987654321", name="James Bond"),
        url="https://sis.gov.uk/agents/007",
    )
    .json()
    .encode("utf-8")
)


async def test_me():
    async with start_test_app() as app:
        await app.produce_raw("my_topic_3", raw_msg)

        def _on_delivery(msg: KafkaMessage, *args):
            logger.warning("me so cool")

        # we don't need to wait for it
        app.produce_raw("my_topic_3", raw_msg, on_delivery=_on_delivery)


asyncio.run(test_me())


print("ok")

[INFO] fast_kafka_api.asyncapi: Async specifications generated at: '/tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api.asyncapi: Async docs generated at '/tmp/000_FastKafkaAPI/asyncapi/docs'
[INFO] fast_kafka_api.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag /tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml @asyncapi/html-template -o /tmp/000_FastKafkaAPI/asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/000_FastKafkaAPI/asyncapi/docs.


[INFO] __main__: consumers_async_loop(): Kafka admin created <confluent_kafka.admin.AdminClient object>.
[DEBUG] fast_kafka_api.confluent_kafka: create_missing_topics(['my_topic_1', 'my_topic_2', 'my_topic_3', 'my_topic_4', 'my_topic_error']): existing_topics=['my_topic_error', 'something_random_2', 'my_topic_1', 'something_random', '__consumer_offsets', 'B', 'C', 'training_data', 'prediction_status', 'realitime_data', 'training_request', 'training_status', 'model_metrics', 'training_mo

In [ ]:
# |export


@patch
def test_run(self: FastKafkaAPI, f: Callable[[], Any], timeout: int = 30):
    async def _loop(app: FastKafkaAPI = self, f: Callable[[], Any] = f):
        logger.info(f"test_run(): starting")
        try:
            async with anyio.create_task_group() as tg:
                with anyio.move_on_after(timeout) as scope:
                    app._on_startup()  # type: ignore

                    if iscoroutinefunction(f):
                        logger.info(f"test_run({app=}, {f=}): Calling coroutine {f}")
                        retval = await f()
                    else:
                        logger.info(f"test_run({app=}, {f=}): Calling function {f}")
                        retval = await asyncer.asyncify(f)()

                return retval
        except Exception as e:
            logger.error(f"test_run(): exception caugth {e}")
            raise e
        finally:
            logger.info(f"test_run({app=}, {f=}): shutting down the app")
            await app._on_shutdown()
            logger.info(f"test_run({app=}, {f=}): finished")

    return asyncer.runnify(_loop)()

In [ ]:
retval = app.test_run(lambda: print("Hello world"))
print(f"{retval=}")

[INFO] __main__: test_run(): starting
[INFO] fast_kafka_api.asyncapi: Async specifications generated at: '/tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api.asyncapi: Async docs generated at '/tmp/000_FastKafkaAPI/asyncapi/docs'
[INFO] fast_kafka_api.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag /tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml @asyncapi/html-template -o /tmp/000_FastKafkaAPI/asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/000_FastKafkaAPI/asyncapi/docs.


[INFO] __main__: consumers_async_loop(): Kafka admin created <confluent_kafka.admin.AdminClient object>.
[DEBUG] fast_kafka_api.confluent_kafka: create_missing_topics(['my_topic_1', 'my_topic_2', 'my_topic_3', 'my_topic_4', 'my_topic_error']): existing_topics=['my_topic_error', 'something_random_2', 'my_topic_1', 'something_random', '__consumer_offsets', 'B', 'C', 'training_data', 'prediction_status', 'realitime_data', 'training_request', 'training_

In [ ]:
# |export


@patch
@asynccontextmanager
async def testing_ctx(self: FastKafkaAPI, timeout: int = 30):
    logger.info(f"test_context(): starting")
    try:
        async with anyio.create_task_group() as tg:
            with anyio.move_on_after(timeout) as scope:
                self._on_startup()  # type: ignore

                yield

    except Exception as e:
        logger.error(f"test_context(): exception caugth {e}")
        raise e
    finally:
        logger.info(f"test_context({self=}): shutting down the app")
        await self._on_shutdown()
        logger.info(f"test_context({self=}): finished")

In [ ]:
async with app.testing_ctx():
    print("app is up and running")
    await anyio.sleep(2)
print("app is shuted down")

[INFO] __main__: test_context(): starting
[INFO] fast_kafka_api.asyncapi: Async specifications generated at: '/tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api.asyncapi: Async docs generated at '/tmp/000_FastKafkaAPI/asyncapi/docs'
[INFO] fast_kafka_api.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag /tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml @asyncapi/html-template -o /tmp/000_FastKafkaAPI/asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/000_FastKafkaAPI/asyncapi/docs.


[INFO] __main__: consumers_async_loop(): Kafka admin created <confluent_kafka.admin.AdminClient object>.
[DEBUG] fast_kafka_api.confluent_kafka: create_missing_topics(['my_topic_1', 'my_topic_2', 'my_topic_3', 'my_topic_4', 'my_topic_error']): existing_topics=['my_topic_error', 'something_random_2', 'my_topic_1', 'something_random', '__consumer_offsets', 'B', 'C', 'training_data', 'prediction_status', 'realitime_data', 'training_request', 'train

In [ ]:
async with app.testing_ctx(timeout=3):
    print("app is up and running")
    await anyio.sleep(1000)
print("app is shuted down")

[INFO] __main__: test_context(): starting
[INFO] fast_kafka_api.asyncapi: Async specifications generated at: '/tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api.asyncapi: Async docs generated at '/tmp/000_FastKafkaAPI/asyncapi/docs'
[INFO] fast_kafka_api.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag /tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml @asyncapi/html-template -o /tmp/000_FastKafkaAPI/asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/000_FastKafkaAPI/asyncapi/docs.


[INFO] __main__: consumers_async_loop(): Kafka admin created <confluent_kafka.admin.AdminClient object>.
[DEBUG] fast_kafka_api.confluent_kafka: create_missing_topics(['my_topic_1', 'my_topic_2', 'my_topic_3', 'my_topic_4', 'my_topic_error']): existing_topics=['my_topic_error', 'something_random_2', 'my_topic_1', 'something_random', '__consumer_offsets', 'B', 'C', 'training_data', 'prediction_status', 'realitime_data', 'training_request', 'train

In [ ]:
# | eval: false

# uvicorn.run(app, host="0.0.0.0", port=6006)